## **Build CNN Model**

In [11]:
"""

Defines the CNN architecture used in this project.

Architecture (configurable at top):
 - Conv2D (x1 filters, m1 x m1 kernel, ReLU)
 - MaxPooling2D
 - Conv2D (x2 filters, m2 x m2 kernel, ReLU)
 - MaxPooling2D
 - Flatten
 - Dense (x3 units, ReLU)
 - Dropout (d)
 - Output Dense (K units, softmax)

Design choices (brief):
 - ReLU: simple, fast, avoids vanishing gradient for shallow nets.
 - 3x3 kernels: effective and parameter-efficient for images.
 - 32/64 filters: sufficient for feature extraction on 128x128 input without overfitting.
 - Dropout 0.5: moderate regularization for a modest dataset.
"""

# -------------------------
# Architecture parameters
# -------------------------
INPUT_SHAPE = (128, 128, 3)   # matches IMG_SIZE
X1 = 32
M1 = 3
X2 = 64
M2 = 3
X3 = 128
DROPOUT_RATE = 0.5
# -------------------------

In [12]:
from tensorflow.keras import layers, models
from tensorflow.keras import regularizers

In [13]:
def build_basic_cnn(input_shape=INPUT_SHAPE, num_classes=9,
                    x1=X1, m1=M1, x2=X2, m2=M2, x3=X3, d=DROPOUT_RATE):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=x1, kernel_size=(m1, m1), activation='relu',
                            padding='same', input_shape=input_shape, name="conv1"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), name="pool1"))

    model.add(layers.Conv2D(filters=x2, kernel_size=(m2, m2), activation='relu',
                            padding='same', name="conv2"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), name="pool2"))

    model.add(layers.Flatten(name="flatten"))
    model.add(layers.Dense(x3, activation='relu', name="fc1"))
    model.add(layers.Dropout(d, name="dropout"))
    model.add(layers.Dense(num_classes, activation='softmax', name="output"))
    return model

    # model = models.Sequential()
    
    # # First conv block
    # model.add(layers.Conv2D(filters=x1, kernel_size=(m1, m1), activation='relu',
    #                         padding='same', input_shape=input_shape,
    #                         kernel_regularizer=regularizers.l2(1e-4), name="conv1"))
    # model.add(layers.MaxPooling2D(pool_size=(2,2), name="pool1"))
    # model.add(layers.Dropout(d, name="dropout1"))
    
    # # Second conv block
    # model.add(layers.Conv2D(filters=x2, kernel_size=(m2, m2), activation='relu',
    #                         padding='same', kernel_regularizer=regularizers.l2(1e-4), name="conv2"))
    # model.add(layers.MaxPooling2D(pool_size=(2,2), name="pool2"))
    # model.add(layers.Dropout(d, name="dropout2"))
    
    # # Flatten and dense
    # model.add(layers.Flatten(name="flatten"))
    # model.add(layers.Dense(x3, activation='relu', kernel_regularizer=regularizers.l2(1e-4), name="fc1"))
    # model.add(layers.Dropout(d, name="dropout3"))
    
    # model.add(layers.Dense(num_classes, activation='softmax', name="output"))
    # return model

## **Train and Evaluate**

In [14]:
"""
- Loads prepared dataset (data/realwaste_prepared.npz)
- Builds the CNN model (from 3_build_cnn_model)
- Trains for 20 epochs with Adam (default learning rate 1e-3)
- Saves model and training plots to outputs/
- Evaluates on test set: test accuracy, confusion matrix, precision, recall
- Saves evaluation results to outputs/

Edit the top variables to change hyperparameters.
"""

# -------------------------
# User-editable globals
# -------------------------
DATA_PREPARED = "../data/realwaste_prepared.npz"
IMG_SIZE = 128
BATCH_SIZE = 32   # global batch size; change here to affect training
EPOCHS = 20
LEARNING_RATE = 1e-3  # chosen for Adam
OUTPUTS_DIR = "../outputs"
# -------------------------

In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential

In [16]:
def load_data(npz_path):
    data = np.load(npz_path, allow_pickle=True)
    X_train = data['X_train']
    y_train = data['y_train']
    X_val = data['X_val']
    y_val = data['y_val']
    X_test = data['X_test']
    y_test = data['y_test']
    classes = data['classes'].tolist()
    class_weights = data['class_weights'].item()
    return X_train, y_train, X_val, y_val, X_test, y_test, classes, class_weights

In [17]:
def plot_history(history, out_png):
    h = history.history
    epochs = range(1, len(h['loss']) + 1)
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(epochs, h['loss'], label='train loss')
    plt.plot(epochs, h['val_loss'], label='val loss')
    plt.xlabel('Epoch'); plt.title('Loss'); plt.legend()
    plt.subplot(1,2,2)
    plt.plot(epochs, h['accuracy'], label='train acc')
    plt.plot(epochs, h['val_accuracy'], label='val acc')
    plt.xlabel('Epoch'); plt.title('Accuracy'); plt.legend()
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()

In [18]:
def pretty_save_text(path, text):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)

In [19]:
def train():
    print("Loading prepared dataset:", DATA_PREPARED)
    X_train, y_train, X_val, y_val, X_test, y_test, classes, class_weights = load_data(DATA_PREPARED)

    input_shape = X_train.shape[1:]
    num_classes = len(classes)
    print("Input shape:", input_shape, "Num classes:", num_classes)

    model = build_basic_cnn(input_shape=input_shape, num_classes=num_classes)
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    model.summary()

    # callbacks
    es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    # train_augmenter = Sequential([
    #     layers.RandomFlip("horizontal"),
    #     layers.RandomRotation(0.1),
    #     layers.RandomZoom(0.1),
    #     layers.RandomContrast(0.1)
    # ])

    # # Apply augmentation
    # X_train_aug = train_augmenter(X_train)


    print(f"Training for {EPOCHS} epochs with batch size {BATCH_SIZE} and Adam lr={LEARNING_RATE}")
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        class_weight=class_weights,
        callbacks=[es, rlrop],
        verbose=2
    )

    # history = model.fit(
    #     X_train_aug, y_train,
    #     validation_data=(X_val, y_val),
    #     epochs=EPOCHS,
    #     batch_size=BATCH_SIZE,
    #     class_weight=class_weights,
    #     callbacks=[es, rlrop],
    #     verbose=2
    # )


    # save model and history
    model_path = os.path.join(OUTPUTS_DIR, "realwaste_cnn_adam.h5")
    model.save(model_path)
    print("Saved model to", model_path)
    np.savez_compressed(os.path.join(OUTPUTS_DIR, "history_adam.npz"), **history.history)

    # plots
    plot_history(history, os.path.join(OUTPUTS_DIR, "train_val_loss_acc_adam.png"))
    print("Saved training plot to outputs/")

    # Evaluate on test
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}  Test Accuracy: {acc:.4f}")

    # predictions
    preds = model.predict(X_test)
    y_pred = np.argmax(preds, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # confusion matrix & classification report
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap="Blues")
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix")
    plt.xticks(rotation=45)
    plt.tight_layout()
    cm_path = os.path.join(OUTPUTS_DIR, "confusion_matrix_adam.png")
    plt.savefig(cm_path); plt.close()
    print("Saved confusion matrix to", cm_path)

    report = classification_report(y_true, y_pred, target_names=classes, digits=4)
    print("\nClassification report:\n", report)

    prec_macro = precision_score(y_true, y_pred, average='macro')
    rec_macro = recall_score(y_true, y_pred, average='macro')

    metrics_text = (
        f"Test accuracy: {acc:.4f}\n"
        f"Test loss: {loss:.4f}\n"
        f"Macro precision: {prec_macro:.4f}\n"
        f"Macro recall: {rec_macro:.4f}\n\n"
        f"Classification report:\n{report}\n"
    )
    pretty_save_text(os.path.join(OUTPUTS_DIR, "evaluation_adam.txt"), metrics_text)
    print("Saved evaluation summary to outputs/evaluation_adam.txt")


In [20]:
if __name__ == "__main__":
    train()

Loading prepared dataset: ../data/realwaste_prepared.npz
Input shape: (128, 128, 3) Num classes: 9


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling2D)            │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool2 (MaxPooling2D)            │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 65536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 128)            │     8,388,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,409,289 (32.08 MB)

 Trainable params: 8,409,289 (32.08 MB)

 Non-trainable params: 0 (0.00 B)

Training for 20 epochs with batch size 32 and Adam lr=0.001
Epoch 1/20
104/104 - 32s - 306ms/step - accuracy: 0.1639 - loss: 2.2536 - val_accuracy: 0.1823 - val_loss: 2.0049 - learning_rate: 0.0010
Epoch 2/20
104/104 - 30s - 293ms/step - accuracy: 0.3458 - loss: 1.7409 - val_accuracy: 0.4334 - val_loss: 1.6095 - learning_rate: 0.0010
Epoch 3/20
104/104 - 30s - 289ms/step - accuracy: 0.4474 - loss: 1.4810 - val_accuracy: 0.4727 - val_loss: 1.4097 - learning_rate: 0.0010
Epoch 4/20
104/104 - 29s - 281ms/step - accuracy: 0.4997 - loss: 1.3308 - val_accuracy: 0.4937 - val_loss: 1.3949 - learning_rate: 0.0010
Epoch 5/20
104/104 - 27s - 262ms/step - accuracy: 0.5331 - loss: 1.2193 - val_accuracy: 0.5288 - val_loss: 1.2960 - learning_rate: 0.0010
Epoch 6/20
104/104 - 28s - 270ms/step - accuracy: 0.5830 - loss: 1.0896 - val_accuracy: 0.5610 - val_loss: 1.2412 - learning_rate: 0.0010
Epoch 7/20
104/104 - 28s - 271ms/step - accuracy: 0.6167 - loss: 0.9621 - val_accuracy: 0.5694 - val_loss: 1.259

Saved model to ../outputs\realwaste_cnn_adam.h5
Saved training plot to outputs/
Test Loss: 1.2198  Test Accuracy: 0.5680
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 104ms/step
Saved confusion matrix to ../outputs\confusion_matrix_adam.png

Classification report:
                      precision    recall  f1-score   support

          Cardboard     0.6429    0.5217    0.5760        69
      Food Organics     0.6441    0.6129    0.6281        62
              Glass     0.6761    0.7619    0.7164        63
              Metal     0.5241    0.6441    0.5779       118
Miscellaneous Trash     0.3765    0.4267    0.4000        75
              Paper     0.7407    0.5333    0.6202        75
            Plastic     0.5794    0.4493    0.5061       138
      Textile Trash     0.3333    0.3125    0.3226        48
         Vegetation     0.6374    0.8923    0.7436        65

           accuracy                         0.5680       713
          macro avg     0.5727    0.5727    0.5657       713
       weighted 

## **Optimizer Comparison**

In [21]:
"""
- Loads prepared data
- Builds the same CNN architecture three times
- Trains each with: Adam (lr=1e-3), SGD (lr=1e-2), SGD+Momentum (lr=1e-2, momentum=0.9)
- Runs 20 epochs each (or stops early with EarlyStopping)
- Saves a comparison plot and writes test accuracies to outputs/optimizer_comparison.txt

Notes:
- Keep BATCH_SIZE consistent with other scripts (edit variable below).
"""

# -------------------------
# Globals
# -------------------------
DATA_PREPARED = "../data/realwaste_prepared.npz"
BATCH_SIZE = 32
EPOCHS = 20
OUTPUTS_DIR = "../outputs"
# -------------------------

In [22]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score

In [23]:
def load_data():
    d = np.load(DATA_PREPARED, allow_pickle=True)
    return d['X_train'], d['y_train'], d['X_val'], d['y_val'], d['X_test'], d['y_test'], d['classes'], d['class_weights'].item()


In [24]:
def train_with_optimizer(opt_name, optimizer, X_train, y_train, X_val, y_val, class_weights):
    num_classes = y_train.shape[1]
    input_shape = X_train.shape[1:]
    model = build_basic_cnn(input_shape=input_shape, num_classes=num_classes)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
    hist = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                     epochs=EPOCHS, batch_size=BATCH_SIZE,
                     class_weight=class_weights, callbacks=[es], verbose=2)
    return model, hist

In [25]:
def pretty_save_text(path, text):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)

In [26]:
def main():
    X_train, y_train, X_val, y_val, X_test, y_test, classes, class_weights = load_data()
    # prepare optimizers
    optimizers = {
        "Adam": Adam(learning_rate=1e-3),
        "SGD": SGD(learning_rate=1e-2),
        "SGD+Momentum": SGD(learning_rate=1e-2, momentum=0.9)
    }

    histories = {}
    models = {}
    test_accs = {}

    for name, opt in optimizers.items():
        print("\n--- Training with", name)
        model, hist = train_with_optimizer(name, opt, X_train, y_train, X_val, y_val, class_weights)
        models[name] = model
        histories[name] = hist.history
        # evaluate on test
        preds = model.predict(X_test)
        y_pred = np.argmax(preds, axis=1)
        y_true = np.argmax(y_test, axis=1)
        acc = accuracy_score(y_true, y_pred)
        test_accs[name] = acc
        # save model for later inspection
        model.save(os.path.join(OUTPUTS_DIR, f"model_{name.replace('+','_')}.h5"))
        print(f"{name} test accuracy: {acc:.4f}")

    # plot validation accuracy curves for comparison
    plt.figure(figsize=(8,6))
    for name, h in histories.items():
        plt.plot(h['val_accuracy'], label=name)
    plt.xlabel("Epoch"); plt.ylabel("Validation Accuracy")
    plt.title("Optimizer comparison (Validation Accuracy)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUTS_DIR, "optimizer_val_acc_comparison.png"))
    plt.close()
    print("Saved optimizer comparison plot to outputs/optimizer_val_acc_comparison.png")

    # save summary text
    text = "Optimizer comparison results (test accuracy):\n"
    for k, v in test_accs.items():
        text += f"{k}: {v:.4f}\n"
    pretty_save_text(os.path.join(OUTPUTS_DIR, "optimizer_comparison.txt"), text)
    print("Saved summary to outputs/optimizer_comparison.txt")

In [27]:
if __name__ == "__main__":
    main()


--- Training with Adam


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
104/104 - 33s - 314ms/step - accuracy: 0.2138 - loss: 2.1114 - val_accuracy: 0.3394 - val_loss: 1.8214
Epoch 2/20
104/104 - 27s - 256ms/step - accuracy: 0.3500 - loss: 1.7556 - val_accuracy: 0.4418 - val_loss: 1.6715
Epoch 3/20
104/104 - 28s - 265ms/step - accuracy: 0.4020 - loss: 1.6071 - val_accuracy: 0.4081 - val_loss: 1.6019
Epoch 4/20
104/104 - 27s - 261ms/step - accuracy: 0.4456 - loss: 1.4454 - val_accuracy: 0.4712 - val_loss: 1.4705
Epoch 5/20
104/104 - 27s - 256ms/step - accuracy: 0.5045 - loss: 1.2629 - val_accuracy: 0.4979 - val_loss: 1.3558
Epoch 6/20
104/104 - 27s - 264ms/step - accuracy: 0.5646 - loss: 1.1382 - val_accuracy: 0.5330 - val_loss: 1.3246
Epoch 7/20
104/104 - 27s - 263ms/step - accuracy: 0.5962 - loss: 1.0175 - val_accuracy: 0.5147 - val_loss: 1.3747
Epoch 8/20
104/104 - 29s - 279ms/step - accuracy: 0.6293 - loss: 0.9236 - val_accuracy: 0.5189 - val_loss: 1.3833
Epoch 9/20
104/104 - 29s - 275ms/step - accuracy: 0.6804 - loss: 0.7910 - val_accuracy: 

Adam test accuracy: 0.5512

--- Training with SGD


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
104/104 - 25s - 241ms/step - accuracy: 0.1392 - loss: 2.1909 - val_accuracy: 0.2104 - val_loss: 2.1690
Epoch 2/20
104/104 - 19s - 184ms/step - accuracy: 0.1744 - loss: 2.1377 - val_accuracy: 0.2777 - val_loss: 2.0759
Epoch 3/20
104/104 - 18s - 174ms/step - accuracy: 0.2189 - loss: 2.0766 - val_accuracy: 0.2847 - val_loss: 2.0413
Epoch 4/20
104/104 - 19s - 184ms/step - accuracy: 0.2390 - loss: 1.9976 - val_accuracy: 0.3717 - val_loss: 1.8883
Epoch 5/20
104/104 - 21s - 198ms/step - accuracy: 0.2793 - loss: 1.9369 - val_accuracy: 0.3703 - val_loss: 1.8299
Epoch 6/20
104/104 - 21s - 200ms/step - accuracy: 0.3073 - loss: 1.8697 - val_accuracy: 0.4404 - val_loss: 1.7156
Epoch 7/20
104/104 - 20s - 195ms/step - accuracy: 0.3416 - loss: 1.7861 - val_accuracy: 0.4011 - val_loss: 1.7226
Epoch 8/20
104/104 - 22s - 209ms/step - accuracy: 0.3569 - loss: 1.7250 - val_accuracy: 0.4797 - val_loss: 1.5978
Epoch 9/20
104/104 - 20s - 197ms/step - accuracy: 0.3942 - loss: 1.6600 - val_accuracy: 

SGD test accuracy: 0.5666

--- Training with SGD+Momentum


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
104/104 - 41s - 393ms/step - accuracy: 0.1425 - loss: 2.1653 - val_accuracy: 0.2833 - val_loss: 2.0169
Epoch 2/20
104/104 - 40s - 382ms/step - accuracy: 0.2111 - loss: 2.0742 - val_accuracy: 0.2020 - val_loss: 2.0731
Epoch 3/20
104/104 - 41s - 393ms/step - accuracy: 0.2195 - loss: 2.0412 - val_accuracy: 0.3212 - val_loss: 1.8657
Epoch 4/20
104/104 - 29s - 276ms/step - accuracy: 0.2850 - loss: 1.9117 - val_accuracy: 0.3885 - val_loss: 1.7740
Epoch 5/20
104/104 - 33s - 319ms/step - accuracy: 0.3082 - loss: 1.8520 - val_accuracy: 0.2973 - val_loss: 1.9758
Epoch 6/20
104/104 - 33s - 314ms/step - accuracy: 0.3241 - loss: 1.8391 - val_accuracy: 0.3268 - val_loss: 1.8003
Epoch 7/20
104/104 - 27s - 263ms/step - accuracy: 0.3731 - loss: 1.6864 - val_accuracy: 0.3773 - val_loss: 1.6392
Epoch 8/20
104/104 - 26s - 247ms/step - accuracy: 0.3434 - loss: 1.7643 - val_accuracy: 0.3941 - val_loss: 1.6254
Epoch 9/20
104/104 - 21s - 206ms/step - accuracy: 0.3824 - loss: 1.6574 - val_accuracy: 

SGD+Momentum test accuracy: 0.6073
Saved optimizer comparison plot to outputs/optimizer_val_acc_comparison.png
Saved summary to outputs/optimizer_comparison.txt
